In [ ]:
%cd ..

In [ ]:
import os
import time
from PIL import Image
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
%matplotlib inline

tf.debugging.set_log_device_placement(False)

os.environ["TFHUB_DOWNLOAD_PROGRESS"] = "True"

dry_run = False

In [ ]:
import tensorflow as tf 
print(tf.test.is_built_with_cuda()) 
print(tf.config.list_physical_devices('GPU'))

In [ ]:
def save_image(image, filename):
    """
    Saves unscaled Tensor Images.
    Args:
      image: 3D image tensor. [height, width, channels]
      filename: Name of the file to save to.
    """
    if not isinstance(image, Image.Image):
        image = tf.clip_by_value(image, 0, 255)
        image = Image.fromarray(tf.cast(image, tf.uint8).numpy())
    image.save(f'{filename}.jpg')
    print(f'Saved as {filename}.jpg')

In [ ]:
from utils import plot_multiple_images, plot_single_image


plot_multiple_images(tf.random.normal((2, 16, 32, 32, 3)) * 255, 'Gaussian noise')

In [ ]:
from esrgan import ESRGAN
from os import path
from data import CLEAR_IMAGES_DIR, get_dataset


# ds_iter = 

# model = ESRGAN()
# model.load()

# for i in range(3):
X, y = next(iter(get_dataset(path.join(CLEAR_IMAGES_DIR, 'train')).batch(1)))
#     enhanced_batch = model(X)
# plot_multiple_images([tf.expand_dims(X[0], axis=0), tf.expand_dims(X[1], axis=0)], 'Enhanced vs groud truth HR images', cmap='gray', figsize=(6, 6))
plot_single_image(X[0], 'LR image (128, 128)')
plt.figure()
plot_single_image(y[0], 'HR image (256, 256)')

In [ ]:
from tensorflow.keras.layers import LeakyReLU, Dropout, Flatten, Conv2D, Dense
from tensorflow.keras import Model


class Discriminator(Model):
    def __init__(self, input_shape):
        super().__init__()
        self.conv1 = Conv2D(64, (5, 5), strides=(2, 2), padding='same', input_shape=input_shape)
        self.lrelu1 = LeakyReLU()
        self.dropout1 = Dropout(0.3)
        
        self.conv2 = Conv2D(128, (5, 5), strides=(2, 2), padding='same')
        self.lrelu2 = LeakyReLU()
        self.dropout2 = Dropout(0.3)
        
        self.conv3 = Conv2D(64, (3, 3), strides=(2, 2), padding='same')
        self.lrelu3 = LeakyReLU()
        
        self.conv4 = Conv2D(32, (3, 3), strides=(2, 2), padding='same')
        self.lrelu4 = LeakyReLU()
        
        self.flatten = Flatten()
        self.dense = Dense(1, activation=None)
    
    def __call__(self, X, training=True):
        outputs = self.conv1(X)
        outputs = self.lrelu1(outputs)
        outputs = self.dropout1(outputs, training=training)
        
        outputs = self.conv2(outputs)
        outputs = self.lrelu2(outputs)
        outputs = self.dropout2(outputs, training=training)

        outputs = self.conv3(outputs)
        outputs = self.lrelu3(outputs)
        
        outputs = self.conv4(outputs)
        outputs = self.lrelu4(outputs)
        
#         print(outputs.shape)

        return self.dense(self.flatten(outputs))
    
    
    def call(self, inputs, training=True):
        return self(inputs, training=training)

    
def test_discriminator():
    batch_size = 8
    inp_shape = batch_size, 256, 256, 3
    d = Discriminator(input_shape=inp_shape)

    assert d(tf.ones(inp_shape)).shape == (batch_size, 1)
    print('Correct discriminator outputs.')
    

test_discriminator()

In [ ]:
class TunedESRGAN(Model):
    def __init__(self, input_shape, *args, **kwargs):
        super().__init__(*args, **kwargs)
        
        self.base_model = hub.KerasLayer('https://tfhub.dev/captain-pool/esrgan-tf2/1', input_shape=input_shape, trainable=True)
        
    def __call__(self, X, training=True):
        return self.base_model(X, training=training)
    
    def call(self, inputs, training=True):
        return self(inputs, training=training)


In [ ]:
# from core.pan import PixelAttentionSRNetwork
from itertools import cycle

    
# # b = SCPA(3)
# # b(tf.ones((8, 64, 64, 3))).shape
batch_size = 32
inp_shape = (batch_size, 64, 64, 3)
# net = PixelAttentionSRNetwork(feat_extr_n_filters=30, upsamp_n_filters=20, n_blocks=16, scale=4, input_shape=inp_shape)
# net.build(inp_shape)
# # print(net(tf.ones(inp_shape)).shape)
# net.summary()

In [ ]:
# gan = TunedESRGAN(inp_shape)
# gan.build(inp_shape)
# gan.summary()

In [ ]:
# import time


# t = time.time()
# gan(tf.ones(inp_shape))

# print('Single forward step for ESRGAN took', time.time() - t) # 16.393023014068604

In [ ]:
# t = time.time()
# net(tf.ones(inp_shape))

# print('Single forward step for PAN took', time.time() - t) # 3.076345920562744

In [ ]:
cross_entropy_loss = tf.keras.losses.BinaryCrossentropy()
sigmoid = tf.keras.activations.sigmoid


# def discriminator_loss(real_outputs, fake_outputs):
#     return (cross_entropy_loss(tf.zeros_like(fake_outputs), fake_outputs) + 
#             cross_entropy_loss(tf.ones_like(real_outputs), real_outputs))


# def generator_loss(fake_outputs):
#     return cross_entropy_loss(tf.ones_like(fake_outputs), fake_outputs)


def discriminator_loss(real_outputs, fake_outputs):
    """Relativistic discriminator loss"""
    # both real_outputs and fake_outputs are logits(non-transformed discriminator output)
    # real data is more realistic than fake data 
    real_realistics_coeff = sigmoid(real_outputs - tf.reduce_mean(fake_outputs))
    # fake data is less realistic than real data 
    fake_realistics_coeff = sigmoid(fake_outputs - tf.reduce_mean(real_outputs))
    
    return (cross_entropy_loss(tf.zeros_like(fake_outputs), real_realistics_coeff) + 
            cross_entropy_loss(tf.ones_like(real_outputs), fake_realistics_coeff))


def generator_loss(real_outputs, fake_outputs):
    """Relativistic generator loss"""
    # real data is more realistic than fake data 
    real_realistics_coeff = sigmoid(real_outputs - tf.reduce_mean(fake_outputs))
    # fake data is less realistic than real data 
    fake_realistics_coeff = sigmoid(fake_outputs - tf.reduce_mean(real_outputs))
    
    return (cross_entropy_loss(tf.ones_like(fake_outputs), real_realistics_coeff) + 
            cross_entropy_loss(tf.zeros_like(real_outputs), fake_realistics_coeff))


In [ ]:
from os import path
from data import CLEAR_IMAGES_DIR, LR_IMAGE_SIZE, HR_IMAGE_SIZE
from utils import plot_multiple_images, plot_single_image
from core.pan import PixelAttentionSRNetwork
from itertools import cycle


input_lr_shape = (None, *LR_IMAGE_SIZE, 3)
input_hr_shape = (None, *HR_IMAGE_SIZE, 3)

cpu = '/CPU:0'
# gpu = '/GPU:0'

device = cpu

max_ckpt_to_keep = 3
checkpoint_dir = './model/x2_checkpoints'
checkpoint_name = 'pix-att-net-ckpt'


with tf.device(device):
    test_dataset = cycle(get_dataset(path.join(CLEAR_IMAGES_DIR, 'train')).batch(4))
    
    sr_net = PixelAttentionSRNetwork(input_lr_shape, feat_extr_n_filters=30, upsamp_n_filters=20, n_blocks=16, scale=2)
    
    checkpoint = tf.train.Checkpoint(sr_net=sr_net)

    sr_net.build(input_lr_shape)

    ckpt_manager = tf.train.CheckpointManager(checkpoint, checkpoint_dir, max_ckpt_to_keep, checkpoint_name=checkpoint_name)

    # try restoring previous checkpoint or initialize a new one
    restored_ckpt_path = ckpt_manager.restore_or_initialize()
    if restored_ckpt_path:
        print('Restored state from checkpoint {}'.format(repr(restored_ckpt_path)))



In [ ]:
X, y = next(test_dataset)
plot_multiple_images([X, np.ones(X.shape) * 255], figsize=(6, 6), title=f'Source images')

print('Max possible psnr', round(float(tf.reduce_mean(tf.image.psnr(y, y, 255))), 4))

y_hat = tf.image.resize(X, HR_IMAGE_SIZE, method='bilinear')
psnr = round(float(tf.reduce_mean(tf.image.psnr(y, y_hat, 255))), 4)
plot_multiple_images([y_hat, y], figsize=(6, 6), title=f'Bilinear inter PSNR: {psnr}')
# plot_multiple_images([y_hat, y], figsize=(6, 6))
# plt.savefig('./x4/bilinear_inter_results.png')


y_hat = tf.image.resize(X, HR_IMAGE_SIZE, method='bicubic')
psnr = round(float(tf.reduce_mean(tf.image.psnr(y, y_hat, 255))), 4)
plot_multiple_images([y_hat, y], figsize=(6, 6), title=f'Bicubic inter PSNR: {psnr}')
# plot_multiple_images([y_hat, y], figsize=(6, 6))
# plt.savefig('./x4/bicubic_inter_results.png')

y_hat = tf.clip_by_value(sr_net(X, training=False), 0, 255)
psnr = round(float(tf.reduce_mean(tf.image.psnr(y, y_hat, 255))), 4)
msssim = round(float(tf.reduce_mean(tf.image.ssim_multiscale(y_hat, y, 255))), 4)
# print(msssim)
plot_multiple_images([y_hat, y], figsize=(6, 6), title=f'Model PSNR: {psnr}. MS-SSIM: {msssim}')
# plot_multiple_images([y_hat, y], figsize=(6, 6))
# plt.savefig('./x4/sr_model_inter_results.png')

# model = ESRGAN()
# model.load()

# y_hat = tf.clip_by_value(model(X), 0, 255)
# psnr = round(float(tf.reduce_mean(tf.image.psnr(y, y_hat, 255))), 4)
# plot_multiple_images([y_hat, y], figsize=(6, 6), title=f'Bilinear inter PSNR: {psnr}')

In [ ]:
# psnr_test_model = gen
from functools import partial

# partial()

# psnr_test_model = partial(tf.image.resize, size=HR_IMAGE_SIZE, method='bilinear')
# psnr_test_model = partial(tf.image.resize, size=HR_IMAGE_SIZE, method='bicubic')
psnr_test_model = partial(sr_net, training=False)

In [ ]:
mean_batch_psnr = 0.0
mean_batch_ssim = 0.0
mean_batch_msssim = 0.0

with tf.device('/GPU:0'):
    batch_size = 50
    dataset = get_dataset(path.join(CLEAR_IMAGES_DIR, 'test'))
    n_batches = 0
    for batch_x, batch_y in dataset.batch(batch_size):
        y_hat = psnr_test_model(batch_x)
        mean_batch_psnr += tf.reduce_mean(tf.image.psnr(y_hat, batch_y, 255))
        mean_batch_ssim += tf.reduce_mean(tf.image.ssim(y_hat, batch_y, 255))
        mean_batch_msssim += tf.reduce_mean(tf.image.ssim_multiscale(y_hat, batch_y, 255))
        n_batches += 1

mean_batch_psnr /= n_batches
mean_batch_ssim /= n_batches
mean_batch_msssim /= n_batches
print('PSNR:', round(float(mean_batch_psnr), 4))
print('SSIM:', round(float(mean_batch_ssim), 4))
print('MSSSIM:', round(float(mean_batch_msssim), 4))